<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [329]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [330]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox

import che
import numpy as np
from scipy.optimize import root

In [331]:
p=che.Props(['Benzene','Toluene'])

In [357]:
d=DotDict()

d.P = 150e3
d.N = 3
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.5,0.5])
d.FT = 370.
d.D = d.F/2
d.B = d.F - d.D

d.F_stage = 2

d.reflux_ratio = 3.


In [358]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [359]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/2 + np.mean(d.bp)
bp_minmax = np.min(d.bp), np.max(d.bp)

# Bug in root solver: convergence failure if T_guess is exactly linearly spaced
d.T = RangeArray(np.round(np.linspace(*bp_scaled,d.N)), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

d.L = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N-1), 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N-1), 0., 2*d.F*d.reflux_ratio)

d.L = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N), 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [360]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [372]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]*p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = np.array(p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:]))
    r.EB[d.F_stage-1] += np.array(p.Hl(d.F*d.Fz, d.FT))
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [373]:
eqs_wrapped = wrap(eqs)

In [374]:
eqs(d2)

(array([ 1.23624021e+05,  1.18679957e+04,  1.59269994e+04,  1.03993720e+04,
         1.76241227e+04, -2.12751350e+03,  3.56405172e+00, -5.69911577e+00,
        -2.67368852e+00,  2.76658442e+01,  3.02367632e+00, -8.38887139e+00,
        -7.90665484e+05,  9.09765714e+05, -8.35032277e+04, -7.28168590e+03,
        -2.43843957e-01,  6.32330681e+06]),
 {'CONDENSER': array([6323306.80566951]),
  'EB': array([-790665.48439603,  909765.71356435,  -83503.22774035]),
  'EQUIL': DeviceArray([[123624.0210907 ,  11867.99569158],
               [ 15926.99943136,  10399.37198186],
               [ 17624.12274311,  -2127.51350008]], dtype=float64),
  'Lin': array([[124.35997646,  30.93972231],
         [110.06344917,  42.45206324],
         [125.22177677, 103.36897257]]),
  'Lout': array([[110.06344917,  42.45206324],
         [125.22177677, 103.36897257],
         [ 20.05431325,  29.94568675]]),
  'MB': array([[ 3.56405172, -5.69911577],
         [-2.67368852, 27.66584425],
         [ 3.02367632, -8.3

In [ ]:
sol=root(eqs_wrapped, x)
print(sol)
solx=sol.x

In [376]:
dunk=xtodunk(solx)
dunk

{'L': array([150.        , 149.98397172, 255.83763633]),
 'Lx': array([[0.55946779, 0.44053221],
        [0.39647357, 0.60352643],
        [0.24731769, 0.75268231]]),
 'QB': array(6659741.06970151),
 'QC': array(6257199.68184472),
 'T': array([377.4497328 , 382.47578967, 387.72607074]),
 'TC': array(372.26289506),
 'V': array([200.        , 199.98397171, 205.83763633]),
 'Vy': array([[0.7526823 , 0.2473177 ],
        [0.60777529, 0.39222471],
        [0.43270501, 0.56729499]])}

In [ ]:
eqs(xtod(sol.x,d2))